In [ ]:
import os
import shutil
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from PIL import UnidentifiedImageError
import zipfile
from tensorflow.keras.models import Model
import shutil

# Load the pretrained model
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add your own classification head on top of the ResNet50 base
x = GlobalAveragePooling2D()(model.output)
x = Dense(1024, activation='relu')(x)
num_classes = 3
output = Dense(num_classes, activation='softmax')(x)

# Create the new model with the modified classification head
#as the pretrained has 1000 classes and we need three only three
#otherwise it will give index error issue
modified_model = Model(inputs=model.input, outputs=output)

# Path of my drive where shoe data set is placed
file_path = "/content/drive/MyDrive/shoe_dataset/shoe_boot_sandal"


# Create folders for each predicted class
class_names = ['boot', 'sandal', 'shoe']
#making an outpyr folder in the drive to store sorted image folders
for class_name in class_names:
    output_dir = "/content/drive/MyDrive/sorted_images"
    class_dir = os.path.join(output_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)

# Iterate through the images in the dataset
for root, _, files in os.walk(file_path):
    for file in files:
        image_path = os.path.join(root, file)

        try:
            # Load and preprocess the image
            img = image.load_img(image_path, target_size=(224, 224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)

            # Make predictions using the pretrained model
            predictions = modified_model.predict(x)
            predicted_class_index = np.argmax(predictions)
            predicted_class_name = class_names[predicted_class_index]


            # Move the image to the corresponding class folder
            destination_dir = os.path.join(output_dir, predicted_class_name)
            shutil.move(image_path, destination_dir)

        except (UnidentifiedImageError, OSError):
            print(f"Skipped file: {image_path} - Not a valid image file")


# Clean up the temporary extracted directory
shutil.rmtree(file_path)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 216ms/step
Skipped file: /content/drive/MyDrive/shoe_dataset/shoe_boot_sandal/.DS_Store - Not a valid image file
1/1 [==============================] - 0s 204ms/step
